In [ ]:
import polars as pl
import pickle

In [ ]:
data = pl.read_csv('./data/ramirez2022/RAMIREZ2022.csv', has_header=True, separator=';')

In [ ]:
from urllib.parse import quote
from urllib.request import HTTPError, urlopen

import polars as pl

def to_InChI(ids: str) -> str:
    "Tries to return InChI from input compound name with pubchem or cactus api."
    try:
        url = (
            "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/"
            + quote(ids)
            + "/property/InChI/TXT"
        )
        with urlopen(url) as ans:
            ans = ans.read().decode("utf8").rstrip()
            ans = ans.split("\n")[0]

    except (TypeError, ValueError, HTTPError):
        try:
            url = (
                "http://cactus.nci.nih.gov/chemical/structure/" + quote(ids) + "/inchi"
            )
            with urlopen(url) as ans:
                ans = ans.read().decode("utf8").rstrip()
                ans = ans.split("\n")[0]
        except (TypeError, ValueError, HTTPError):
            print("not ok:", url)
            ans = None
    return ans

In [ ]:
inchis = data['name'].apply(to_InChI).rename('inchi')

In [ ]:
inchis.describe()

In [ ]:
data = data.with_columns(inchis)

In [ ]:
data.write_parquet("./data/ramirez2022/raw/data.parquet")